Ce notebook detaille comment ameliorer l'accuracy d'un entrainement en adoptant differentes approches ,notamment l'augmentation des données,la recherche de la meilleure combinaison d'hypermaretres et en fonction des modeles choisis (le transfert learning).

import

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import ParameterGrid
 

déclaration des variables

In [ ]:
# Définir le chemin du dossier NotMNIST Small
notmnist_path = "notMNIST_small"

# Liste des classes (sous-dossiers dans le dossier NotMNIST Small)
classes = sorted(os.listdir(notmnist_path))

# Charger les images et les étiquettes
images = []
labels = []


Chargement des données depuis le dossier notMnist

In [ ]:
for i, class_name in enumerate(classes):
    class_path = os.path.join(notmnist_path, class_name)
    
    # Ajouter une vérification si le chemin est un répertoire
    if os.path.isdir(class_path):
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = load_img(img_path, color_mode='grayscale', target_size=(28, 28))
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(i)
    else:
        print(f"Ignoring non-directory entry: {class_path}")

# Convertir les listes en tableaux NumPy
images = np.array(images)
labels = np.array(labels)

# Normaliser les valeurs des pixels
images = images.astype('float32') / 255.0

# Convertir les étiquettes en one-hot encoding
labels = to_categorical(labels, len(classes))

repartition des données en donnees de test et d'entrainement

In [ ]:

train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)


creation du model de test

In [ ]:
# Définir l'architecture du modèle
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])
# Compiler le modèle
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


Entrainement du model avec les données initiales

In [ ]:
# Entraîner le modèle sans augmentation des données
model.fit(train_images, train_labels, batch_size=64, epochs=10, validation_data=(test_images, test_labels))

# Évaluer le modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Accuracy on test set: {test_acc}')


Tentative d'amelioration de l'accuracy en exploitant differentes approches,notamment l'augmentation des donnees,la recherche d'hyperparametres optimals 

Augmentation des données 

In [ ]:
datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(train_images)

Entrainement du model avec augmentation de données

In [ ]:
# Entraîner le modèle avec augmentation des données
model.fit(datagen.flow(train_images, train_labels, batch_size=64),
          steps_per_epoch=len(train_images) // 64,  # ajuster en fonction de la taille de votre ensemble d'entraînement
          epochs=10,
          validation_data=(test_images, test_labels))


Si l'accurracy diminue avec l'augmentation cela veut dire l'augmentation est violente et il faut reduire la valeur parametres comme suite 

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=5,           
    horizontal_flip=True
)
datagen.fit(train_images)

In [ ]:
model.fit(datagen.flow(train_images, train_labels, batch_size=64),
          steps_per_epoch=len(train_images) // 64,  # ajuster en fonction de la taille de votre ensemble d'entraînement
          epochs=10,
          validation_data=(test_images, test_labels))
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Accuracy on test set: {test_acc}')

Evaluation sur l'ensemble des données de test 

In [ ]:

# Évaluer le modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Accuracy on test set: {test_acc}')

Definition de la grille d'hyperparametres,pour la recherche de la meilleur combinaison d'hyperparametres optimale

In [ ]:
# Définir le hyperparameter grid
param_grid = {
    'learning_rate': [0.0001, 0.001, 0.01],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30]
}


Generationd de toutes les combinaisons d'hyperparametre possible

In [ ]:
# Generate all combinations of hyperparameters
param_combinations = list(ParameterGrid(param_grid))

recherche iteratif sur notre modele
On aurait pu utiliser d'autre methode tels que gridSearch,randomSearch mais nous voulions vraiment  prendre en compte tous les cas memes si ça reste quand meme laborieux.Avec ce procedé on a reuissi à avoir une accuracy de 96.5% avec les hyperparmetre suivant :
{'batch_size': 32, 'epochs': 30, 'learning_rate': 0.001}


In [ ]:
best_accuracy = 0.0  # Initialiser la meilleure précision
best_params = None  # Initialiser les meilleurs hyperparamètres

for params in param_combinations:
    print(f"Training with hyperparameters: {params}")
    
    # Définir une architecture de modèle plus complexe
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(classes), activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Entraîner le modèle avec augmentation des données
    model.fit(datagen.flow(train_images, train_labels, batch_size=params['batch_size']),
              steps_per_epoch=len(train_images) // params['batch_size'],
              epochs=params['epochs'],
              validation_data=(test_images, test_labels))
    
    # Évaluer le modèle sur l'ensemble de test
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print(f'Accuracy on test set: {test_acc}')
    
    # Mettre à jour les meilleurs hyperparamètres si la performance est meilleure
    if test_acc > best_accuracy:
        best_accuracy = test_acc
        best_params = params

# Imprimer les meilleurs hyperparamètres
print("Best hyperparameters:", best_params)
print("Best accuracy on test set:", best_accuracy)


In [ ]:
# Définir une architecture de modèle plus complexe
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(classes), activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=best_params['learning_rate']),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Entraîner le modèle avec les meilleurs hyperparamètres
model.fit(datagen.flow(train_images, train_labels, batch_size=best_params['batch_size']),
          steps_per_epoch=len(train_images) // best_params['batch_size'],
          epochs=best_params['epochs'],
          validation_data=(test_images, test_labels))

# Évaluer le modèle sur l'ensemble de test
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Accuracy on test set: {test_acc}')